In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

In [5]:
# read hw 1.1 file
df_1 = pd.read_csv("homework_1.1.csv")

In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1000 non-null   int64  
 1   X1          1000 non-null   float64
 2   X2          1000 non-null   float64
 3   X3          1000 non-null   float64
 4   Y           1000 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 39.2 KB


In [7]:
df_1.head(10)

,Unnamed: 0,X1,X2,X3,Y
0,0,-0.440646,-0.390227,0.156718,-0.877671
1,1,-3.810099,-1.304665,-1.105117,-10.130388
2,2,-1.425451,-0.340049,1.115908,0.284068
3,3,-1.325750,0.161906,-0.254670,-1.994344
4,4,3.120263,1.487343,-1.164839,2.030030
5,5,-3.462721,-0.986104,-0.383416,-6.946920
6,6,-0.353195,-0.086630,-0.631641,-2.805805
7,7,-1.612043,-1.042533,0.915845,-0.755447
8,8,-2.119923,-1.030619,0.027932,-4.655447
9,9,-0.520106,-0.662596,1.295298,2.222797


In [8]:
df_1.drop(columns=["Unnamed: 0"])

,X1,X2,X3,Y
0,-0.440646,-0.390227,0.156718,-0.877671
1,-3.810099,-1.304665,-1.105117,-10.130388
2,-1.425451,-0.340049,1.115908,0.284068
3,-1.325750,0.161906,-0.254670,-1.994344
4,3.120263,1.487343,-1.164839,2.030030
...,...,...,...,...
995,1.281625,-0.062024,-0.109231,1.206058
996,-3.047558,-1.121675,0.678439,-3.229262
997,-0.028757,-0.191722,1.191790,3.160897
998,0.415266,0.373086,1.172636,4.033302


In [9]:
X = df_1.drop(columns="Y")
y = df_1["Y"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 2.662e+04
Date:                Thu, 12 Jun 2025   Prob (F-statistic):               0.00
Time:                        00:22:06   Log-Likelihood:                -726.29
No. Observations:                1000   AIC:                             1463.
Df Residuals:                     995   BIC:                             1487.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0476      0.032      1.496      0.1

In [10]:
X1 = sm.add_constant(df_1[['X1']]) 
model_X1 = sm.OLS(df_1['Y'], X1).fit()
print(model_X1.params['X1'])

1.8417610991461832


In [11]:
X2 = sm.add_constant(df_1[['X2']]) 
model_X2 = sm.OLS(df_1['Y'], X2).fit()
print(model_X2.params['X2'])

4.083612579373998


In [12]:
X3 = sm.add_constant(df_1[['X3']]) 
model_X3 = sm.OLS(df_1['Y'], X3).fit()
print(model_X3.params['X3'])

3.0970412020498452


In [13]:
df_2 = pd.read_csv("homework_1.2.csv")

In [14]:
from sklearn.neighbors import NearestNeighbors

In [15]:
df_2.head(5)

,Unnamed: 0,X,Y,Z
0,0,0,0.548814,0.548814
1,1,1,1.215189,0.715189
2,2,0,0.602763,0.602763
3,3,0,0.544883,0.544883
4,4,0,0.423655,0.423655


In [16]:
group_0 = df_2[df_2["X"] == 0][["Z"]].values
group_1 = df_2[df_2["X"] == 1][["Z"]].values

In [17]:
nn = NearestNeighbors(n_neighbors= 1 , metric= "euclidean")
nn.fit(group_1)

NearestNeighbors(metric='euclidean', n_neighbors=1)

In [18]:
import numpy as np

In [19]:
distances, indicate = nn.kneighbors(group_0)
farthest_distance = np.max(distances)


print("Farthest match distance:", farthest_distance)

Farthest match distance: 0.11550108502062191


In [20]:
group_0 = df_2[df_2['X'] == 0].copy()
group_1 = df_2[df_2['X'] == 1].copy()
nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
nn.fit(group_1[['Z']])
distances, indices = nn.kneighbors(group_0[['Z']])
matched_group_1 = group_1.iloc[indices.flatten()]

avg_y_x0 = group_0['Y'].mean()
avg_y_x1_matched = matched_group_1['Y'].mean()

effect = avg_y_x1_matched - avg_y_x0
print("Effect (Y_X=1 - Y_X=0):", effect)

Effect (Y_X=1 - Y_X=0): 0.5197742360609017


In [21]:
# Set parameters
caliper = 0.25  # max allowed distance in Z
k = 3  # max number of closest matches per treated unit

# Fit NearestNeighbors on group_0 (control)
nn = NearestNeighbors(n_neighbors=len(group_0), metric='euclidean')
nn.fit(group_0[['Z']])

matched_y_values = []

# Loop over each treated unit in group_1
for z_value in group_1['Z']:
    distances, indices = nn.kneighbors([[z_value]])
    
    # Select top k matches within caliper distance
    valid_matches = [(dist, idx) for dist, idx in zip(distances[0], indices[0]) if dist <= caliper][:k]
    
    if valid_matches:
        matched_indices = [idx for _, idx in valid_matches]
        matched_y = group_0.iloc[matched_indices]['Y'].values
        matched_y_values.append(np.mean(matched_y))

# Compute estimated effect
avg_y_x0 = group_0['Y'].mean()
avg_y_x1_matched = np.mean(matched_y_values)
effect = avg_y_x1_matched - avg_y_x0

print("Average Y for X = 0:", avg_y_x0)
print("Average matched Y for X = 1:", avg_y_x1_matched)
print("Estimated treatment effect:", effect)


Average Y for X = 0: 0.3317446410426721
Average matched Y for X = 1: 0.569063350135839
Estimated treatment effect: 0.23731870909316688


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature nam